# Create Slice for Fabric Rack
This is the slice creation for the ```Traffic Analysis``` tutorial. This is the first step of the assignment, to create the slice on fabric using fablib.

## 1. Set up the Experiment

In this section you will use the Fablib manager to create a new slice that is composed of 2 nodes for the Traffic Analysis Assignment.

### 1.1 Reserve Resources
In the 'EDUKY' site, we will reserve a set of 2 nodes arranged in a line, with one node designated as a server and the other as a client. Each node will have the following specifications: 1 CPU core, 2GB of RAM, and 10GB of storage capacity. Each node will have 1 network card (NICs) to communicate with each other, on the server we will call the card "server_port" and on the client we will call it "client_port". The bridge connecting the two nodes is an l2network called "lan". All nodes will be preloaded with an 'Ubuntu' Linux OS. Upon submission, the slice will be named 'TCP_Traffic_intro'.

In [ ]:
# Import Fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()                     
fablib.show_config()
import json
import traceback

In [ ]:
# Define and Submit Slice
slice_name = "TrafficAnalysis"
site = "EDUKY"
print(site)

nicmodel = "NIC_Basic"
image = "default_ubuntu_20"

cores = 1
ram = 2
disk = 10
network_name = 'lan'

try:
    #Create Slice
    slice = fablib.new_slice(name = slice_name)
    
    #Server
    server = slice.add_node(name="Server", site=site)
    server.set_image(image)
    server_port = server.add_component(model=nicmodel, name="server_port").get_interfaces()[0] 
    
    #Client
    client = slice.add_node(name="Client", site=site)
    client.set_image(image)
    client_port = client.add_component(model=nicmodel, name="client_port").get_interfaces()[0] 
    
    #changed name
    lan = slice.add_l2network(name=network_name, interfaces=[server_port, client_port])
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

### 1.2 Set up the Experiment Network
The network configuration part of this slice will connect the nodes with specific IPs such that the nodes are able to communicate with each other. The slice will use the network name to find the corresponding devices (NIC cards) used in the slice creation stage so that we can provide the correct connection in the network, in this case, "lan".

This slice contains a single subnetwork in IPv4: 10.1.1.0/24 (between server and the client, "lan"). This cell creates a subnet and pops available IPs into the NIC cards of each of the nodes, adding them along with the subnet.

After providing the IPs to the corresponding devices, the cell will output the network status and execute "ip a" to check the connections that are active.

In [ ]:
# Setup Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet = IPv4Network("10.1.1.0/24")
available_ips = list(subnet)[1:]


network_name = 'lan'

try:
    for node in slice.get_nodes():
        node_iface = node.get_interface(network_name=network_name)  
        node_addr = available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_addr, subnet=subnet)
        
        for interface in node.get_interfaces():
            stdout, stderr = node.execute(f'sudo ip link set dev { interface.get_device_name()} up')
        print(f"Network status at node {node.get_name()}")
        node.execute("ip a")

        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')

except Exception as e:
    print(f"Fail: {e}")

### 1.3 Configure the software needed for the nodes in the experiment
In this section of the slice creation, we will add any additional software, tools, or scripts that we need for our experiments. For this slice, we only need to install "net-tools" and "iperf". We provide the software by running node.execute() and letting the node know to download the software.

In [ ]:
# Install Software
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    for node in slice.get_nodes():
        stdout, stderr = node.execute(f'sudo apt-get update')
        node.execute("sudo apt install net-tools")
        stdout, stderr = node.execute(f'sudo apt-get install iperf')
except Exception as e:
    print(f"Exception: {e}")

## Continue to The experiment notebook

Once you have completed this notebook you should be able to continue to the Traffic Analysis. You can either open it on the explorer or click [Here](./Traffic_Analysis.ipynb) to open the next notebook.